# Testando as funcoes de similaridad no dataframe song_data

In [ ]:
#imports 
import pandas as pd
import numpy as np
import random
import time
from datetime import datetime, timedelta
import pickle 

#dataframe of songs
df = pd.read_csv(
    'song_data.csv', 
     names =['user_id', 'song_id', 'listen_count', 'title', 'artist']
)
df.drop(['title','artist'],1,inplace=True)


In [40]:
all_songs = pd.read_csv(
    'top_songs.csv' 
     )
all_songs.drop('index',1,inplace=True)

def show_song(dataframe,song_id):
    song_df = dataframe.loc[dataframe['song_id'] == song_id]
    artist = "".join(song_df['artist'].astype('str').tail(1).tolist()) 
    title = "".join(song_df['title'].astype('str').tail(1).tolist())
    return (title,artist)


all_songs.head()

,song_id,listen_count,artist,title
0,SOBONKR12A58A7A7E0,40619,Dwight Yoakam,You're The One
1,SOAUWYT12A81C206F1,36059,Björk,Undo
2,SOSXLTC12AF72A7F54,30391,Kings Of Leon,Revelry
3,SOEGIYH12A6D4FC0E3,21953,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
4,SOFRQTD12A81C233C0,21646,Harmonia,Sehr kosmisch


In [19]:
#scrip para criar um dicionario item:{user:score}
dic_songs = {}
start_time = time.time()
for i,song in enumerate(df['song_id']):
    if i%10000==0:
        print('rest:{} '.format(1116609 - i), end=' - ')
    if song not in dic_song.keys():
        dic_songs.setdefault(song,{})
    user = df['user_id'][i]
    dic_songs[song][user] = df['listen_count'][i] 

duration = time.time() - start_time
sec = timedelta(seconds=int(duration))
d_time = datetime(1,1,1) + sec    
print("\n duration:  %d:%d:%d:%d" % (d_time.day-1, d_time.hour, d_time.minute, d_time.second), end='')
print(" (DAYS:HOURS:MIN:SEC)") 


rest:1116609  - rest:1106609  - rest:1096609  - rest:1086609  - rest:1076609  - rest:1066609  - rest:1056609  - rest:1046609  - rest:1036609  - rest:1026609  - rest:1016609  - rest:1006609  - rest:996609  - rest:986609  - rest:976609  - rest:966609  - rest:956609  - rest:946609  - rest:936609  - rest:926609  - rest:916609  - rest:906609  - rest:896609  - rest:886609  - rest:876609  - rest:866609  - rest:856609  - rest:846609  - rest:836609  - rest:826609  - rest:816609  - rest:806609  - rest:796609  - rest:786609  - rest:776609  - rest:766609  - rest:756609  - rest:746609  - rest:736609  - rest:726609  - rest:716609  - rest:706609  - rest:696609  - rest:686609  - rest:676609  - rest:666609  - rest:656609  - rest:646609  - rest:636609  - rest:626609  - rest:616609  - rest:606609  - rest:596609  - rest:586609  - rest:576609  - rest:566609  - rest:556609  - rest:546609  - rest:536609  - rest:526609  - rest:516609  - rest:506609  - rest:496609  - rest:486609  - rest:476609  - rest:466609  

Agora vamos definir as funcoes de similaridade

In [24]:
def sim_distance(prefs,person1,person2):
    intersection = []
    for item in prefs[person1]:
        if item in prefs[person2]:
            intersection.append(item)
    if len(intersection)==0:
        return 0
    sum_of_squares = np.sum([(prefs[person1][item] - prefs[person2][item])**2\
                     for item in intersection])
    return 1/(1+ sum_of_squares)

In [26]:
songs = df['song_id'].unique()
song1,song2 = songs[0],songs[1]
print(sim_distance(dic_songs,song1,song2))

0.5


In [27]:
print(sim_distance(dic_songs,song1,song1))

1.0


In [33]:
def naive_pearson(prefs,person1,person2):
    intersection = []
    for item in prefs[person1]:
        if item in prefs[person2]:
            intersection.append(item)
    number_of_itens = len(intersection) 
    if number_of_itens == 0:
        return 0
    sum1 = np.sum([prefs[person1][item] for item in intersection])/number_of_itens
    sum2 = np.sum([prefs[person2][item] for item in intersection])/number_of_itens
    sum1_mean = np.sum([(prefs[person1][item] - sum1)**2 for item in intersection])
    sum2_mean = np.sum([(prefs[person2][item] - sum2)**2 for item in intersection])
    pSum = np.sum([(prefs[person1][item]- sum1) * (prefs[person2][item]- sum2)\
                     for item in intersection])
    numerator = pSum 
    denominator = np.sqrt(sum1_mean)*np.sqrt(sum2_mean)
    if denominator == 0:
        return 0
    return numerator/denominator

def sim_pearson(prefs,person1,person2):
    intersection = []
    for item in prefs[person1]:
        if item in prefs[person2]:
            intersection.append(item)
    number_of_itens = len(intersection) 
    if number_of_itens ==0:
        return 0
    sum1 = np.sum([prefs[person1][item] for item in intersection])
    sum2 = np.sum([prefs[person2][item] for item in intersection])
    sum1sq = np.sum([(prefs[person1][item])**2 for item in intersection])
    sum2sq = np.sum([(prefs[person2][item])**2 for item in intersection])
    pSum = np.sum([(prefs[person1][item] * prefs[person2][item])\
                     for item in intersection])
    numerator = pSum - (sum1*sum2/number_of_itens)
    denominator = np.sqrt( ((sum1sq - sum1**2)/number_of_itens) *\
                           ((sum2sq - sum2**2)/number_of_itens))
    if denominator == 0:
        return 0
    return numerator/denominator

In [31]:
print(naive_pearson(dic_songs,song1,song2))

0


In [34]:
print(sim_pearson(dic_songs,song1,song2))

0


In [35]:
print(naive_pearson(dic_songs,song1,song1))

1.0


In [36]:
print(sim_pearson(dic_songs,song1,song1))

2.90707549499


Note que a medida sim_person esta passando acima de 1.

In [45]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person,other_person),other_person)\
              for other_person in prefs if other_person!= person]
    scores.sort(reverse=True)
    return scores[0:n]

E quando as mais sememlhantes temos resultados diferentes usando as duas medidas

In [49]:
print("song1 is {}".format(show_song(all_songs,song1)))

song1 is ('The Cove', 'Jack Johnson')


In [43]:
top1 = [y for (x,y) in topMatches(dic_songs,song1)]
show1 = list(map(lambda x: show_song(all_songs,x), top1))
print(show1)

[('The Sharing Song', 'Jack Johnson'), ('Jungle Gym', 'Jack Johnson / G. Love'), ('Lively Up Yourself', 'Bob Marley'), ('Talk Of The Town', 'Jack Johnson / Kawika Kahiapo'), ('Situations', 'Jack Johnson')]


In [47]:
top2 = [y for (x,y) in topMatches(dic_songs,song1,similarity=naive_pearson)]
show2 = list(map(lambda x: show_song(all_songs,x), top2))
print(show2)

[('Howlin\x19 For You', 'The Black Keys'), ('Better Man', 'Pearl Jam'), ('Thank You for Being a Friend', 'Andrew Gold'), ('Danse Pour Moi', 'M. Pokora'), ('Monsoon', 'Jack Johnson')]


In [50]:
top3 = [y for (x,y) in topMatches(dic_songs,song1,similarity=sim_distance)]
show3 = list(map(lambda x: show_song(all_songs,x), top3))
print(show3)

[('Sunset (Bird Of Prey)', 'Fatboy Slim'), ('Watch The Tapes', 'LCD Soundsystem'), ('Opium Of The People (Album Version)', 'Slipknot'), ('Rosemary', 'Scott Walker'), ('Nostalgia', 'The Long Blondes')]


Note que show1 (usando sim_pearson) e o resultado que faz mais sentido

In [51]:
def calculateSimilarItems(itemPrefs,n=10):
    result = {}
    c = 0
    for item in itemPrefs:
        c += 1
        if c%100==0:
            print('%d / %d' % (c,len(itemPrefs)))
        scores = topMatches(itemPrefs, item, n=n, similarity=sim_pearson)
        result[item]=scores
    return result

Agora vamos criar um dicionario song:[top 10 similar songs]

In [53]:
start_time = time.time()

itemsim = calculateSimilarItems(dic_songs)

duration = time.time() - start_time
sec = timedelta(seconds=int(duration))
d_time = datetime(1,1,1) + sec    
print("\n duration:  %d:%d:%d:%d" % (d_time.day-1, d_time.hour, d_time.minute, d_time.second), end='')
print(" (DAYS:HOURS:MIN:SEC)") 

100 / 10000
200 / 10000
300 / 10000
400 / 10000
500 / 10000
600 / 10000
700 / 10000
800 / 10000
900 / 10000
1000 / 10000
1100 / 10000
1200 / 10000
1300 / 10000
1400 / 10000
1500 / 10000
1600 / 10000
1700 / 10000
1800 / 10000
1900 / 10000
2000 / 10000
2100 / 10000
2200 / 10000
2300 / 10000
2400 / 10000
2500 / 10000
2600 / 10000
2700 / 10000
2800 / 10000
2900 / 10000
3000 / 10000
3100 / 10000
3200 / 10000
3300 / 10000
3400 / 10000
3500 / 10000
3600 / 10000
3700 / 10000
3800 / 10000
3900 / 10000
4000 / 10000
4100 / 10000
4200 / 10000
4300 / 10000
4400 / 10000
4500 / 10000
4600 / 10000
4700 / 10000
4800 / 10000
4900 / 10000
5000 / 10000
5100 / 10000
5200 / 10000
5300 / 10000
5400 / 10000
5500 / 10000
5600 / 10000
5700 / 10000
5800 / 10000
5900 / 10000
6000 / 10000
6100 / 10000
6200 / 10000
6300 / 10000
6400 / 10000
6500 / 10000
6600 / 10000
6700 / 10000
6800 / 10000
6900 / 10000
7000 / 10000
7100 / 10000
7200 / 10000
7300 / 10000
7400 / 10000
7500 / 10000
7600 / 10000
7700 / 10000
7800 / 1

In [54]:
#quardando no pickle
f = open('dic_songs.pickle', 'wb')
di ={'dic_songs':dic_songs,'itemsim':itemsim}
pickle.dump(di,f)
f.close()

In [60]:
def get_user_prefs(dataframe, user):
    new_df =  dataframe.loc[dataframe['user_id'] == user]
    new_df = new_df.drop('user_id',1)
    new_df = new_df.set_index('song_id')
    quasi_dic = new_df.to_dict()
    dic = quasi_dic['listen_count']
    return dic


def getRecommendedItems(itemMatch,user,dataframe):
    userRatings = get_user_prefs(dataframe,user)
    scores ={}
    totalSim ={}
    for (item,rating) in userRatings.items():
        for (similarity,item2) in itemMatch[item]:
            if item2 in userRatings:
                continue
            scores.setdefault(item2,0)
            scores[item2] += similarity*rating
            totalSim.setdefault(item2,0)
            totalSim[item2] += similarity
        
    rankings = [(score/totalSim[item],item) for item,score in scores.items()]      
    rankings.sort(reverse=True)
    return rankings

In [59]:
users = df['user_id'].unique()
target = users[0]
target_prefs = get_user_prefs(df,target)
print(target_prefs)

{'SOSYBEV12AB0182933': 1, 'SOXMIUS12A8C13CD59': 1, 'SOIZAZL12A6701C53B': 5, 'SORWLTW12A670208FA': 1, 'SODACBL12A8C13C273': 1, 'SONSAEZ12A8C138D7A': 1, 'SOBBMDR12A8C13253B': 2, 'SOVQEYZ12A8C1379D8': 1, 'SOXZQDE12A8C135833': 1, 'SOWSPUS12AC468BEE3': 1, 'SODXRTY12AB0180F3B': 1, 'SORZASF12A6D4F8CFA': 2, 'SORUFVF12AB018230B': 1, 'SOXRXDG12A8C131DE5': 1, 'SOZOBWN12A8C130999': 1, 'SOTLVCL12AB0182D22': 1, 'SOPCVQE12AC468AF36': 1, 'SOBXHDL12A81C204C0': 1, 'SOFRQTD12A81C233C0': 1, 'SOQIVUD12AB01821D2': 1, 'SOJPFQG12A58A7833A': 1, 'SORPMYJ12AF729EB90': 1, 'SOHQWYZ12A6D4FA701': 1, 'SOFGUAY12AB017B0A8': 1, 'SOQLCKR12A81C22440': 1, 'SOUKXIN12A8C133C7F': 1, 'SOBYHAJ12A6701BF1D': 1, 'SORQHCG12A58A7EEBA': 1, 'SOIYTOA12A6D4F9A23': 1, 'SOVYIYI12A8C138D88': 1, 'SOVHRGF12A8C13852F': 1, 'SOMGIYR12AB0187973': 6, 'SOYHEPA12A8C13097F': 8, 'SOMSQJY12A8C138539': 1, 'SOLLGNU12AF72A4D4F': 1, 'SOAKIMP12A8C130995': 1, 'SOOKGRB12A8C13CD66': 1, 'SOKRIMP12A6D4F5DA3': 5, 'SOMLMKI12A81C204BC': 1, 'SOWQLXP12AF72A08A2': 1,

In [64]:
dft1 = df.loc[df['user_id'] == target]
dft1 = dft1.sort_values('listen_count',ascending=False)
best_songs_t1 = list(map(lambda x: show_song(all_songs,x),list(dft1['song_id'])))
if len(best_songs_t1) > 5:
    print("The user target likes: \n", best_songs_t1[0:5])
else:
    print("The user target likes: \n",best_songs_t1)

The user target likes: 
 [('Moonshine', 'Jack Johnson'), ('Behind The Sea [Live In Chicago]', 'Panic At The Disco'), ("I?'m A Steady Rollin? Man", 'Robert Johnson'), ("Apuesta Por El Rock 'N' Roll", 'Héroes del Silencio'), ("I'll Be Missing You (Featuring Faith Evans & 112)(Album Version)", 'Puff Daddy')]


In [63]:
rec = [y for (x,y) in  getRecommendedItems(itemsim,target,df)]
show4 = list(map(lambda x: show_song(all_songs,x), rec[0:5]))
print(show4)

[("Boy's Life", 'Dr. Know'), ('Deprogram', 'Dr. Know'), ('Rent', 'Pet Shop Boys'), ('Rosemary', 'Scott Walker'), ('Laura', 'Scissor Sisters')]


# Alguns testes

In [73]:
size = len(users)
random_users = random.sample(range(size), 10)

for user in random_users:
    dfuser = df.loc[df['user_id'] == users[user]]
    dfuser = dfuser.sort_values('listen_count',ascending=False)
    best_songs_user = list(map(lambda x: show_song(all_songs,x),list(dfuser['song_id'])))
    if len(best_songs_user) > 3:
        print("\n The user likes: \n", best_songs_user[0:3])
    else:
        print("\n The user likes: \n",best_songs_user) 
    recommendations = [y for (x,y) in  getRecommendedItems(itemsim,users[user],df)]
    show = list(map(lambda x: show_song(all_songs,x), recommendations[0:5]))
    print("And we recommend for him/her: \n", show)




 The user likes: 
 [('Eternal Flame (Single Version)', 'Atomic Kitten'), ('Représente', 'Alliance Ethnik'), ("Le Jardin d'Hiver", 'Jacky Terrasson')]
And we recommend for him/her: 
 [('Flake', 'Jack Johnson'), ('Drops Of Jupiter', 'Train'), ('Here Without You', '3 Doors Down'), ("Go Skate! (Possessed To Skate '97)", 'Suicidal Tendencies'), ('Because You Loved Me', 'Céline Dion')]

 The user likes: 
 [('More Than A Feeling', 'Boston'), ('Evil', 'Interpol'), ('Hoisting The Flag', 'Sondre Lerche')]
And we recommend for him/her: 
 [('Heavy  (LP Version)', 'Collective Soul'), ('Fade To Black', 'Metallica'), ('Paa vauhtii', 'Uniikki'), ('Queen Of Memphis (Dance Mix)', 'Confederate Railroad'), ('Sea Green_ See Blue', 'JayMay')]

 The user likes: 
 [('Revelry', 'Kings Of Leon'), ('Undo', 'Björk'), ('Somebody To Love', 'Justin Bieber')]
And we recommend for him/her: 
 [('Alone In This Bed (Capeside)', 'Framing Hanley'), ("It's Raining Men (Radio Edit)", 'Geri Halliwell'), ('Ray Of Light (Album

# Vamos fazer um outro itemsim com outra medida

In [74]:
def calculateSimilarItems(itemPrefs,n=10):
    result = {}
    c = 0
    for item in itemPrefs:
        c += 1
        if c%100==0:
            print('%d / %d' % (c,len(itemPrefs)))
        scores = topMatches(itemPrefs, item, n=n, similarity=sim_distance)
        result[item]=scores
    return result

start_time = time.time()

itemsim_sim_distance = calculateSimilarItems(dic_songs)

duration = time.time() - start_time
sec = timedelta(seconds=int(duration))
d_time = datetime(1,1,1) + sec    
print("\n duration:  %d:%d:%d:%d" % (d_time.day-1, d_time.hour, d_time.minute, d_time.second), end='')
print(" (DAYS:HOURS:MIN:SEC)") 

100 / 10000
200 / 10000
300 / 10000
400 / 10000
500 / 10000
600 / 10000
700 / 10000
800 / 10000
900 / 10000
1000 / 10000
1100 / 10000
1200 / 10000
1300 / 10000
1400 / 10000
1500 / 10000
1600 / 10000
1700 / 10000
1800 / 10000
1900 / 10000
2000 / 10000
2100 / 10000
2200 / 10000
2300 / 10000
2400 / 10000
2500 / 10000
2600 / 10000
2700 / 10000
2800 / 10000
2900 / 10000
3000 / 10000
3100 / 10000
3200 / 10000
3300 / 10000
3400 / 10000
3500 / 10000
3600 / 10000
3700 / 10000
3800 / 10000
3900 / 10000
4000 / 10000
4100 / 10000
4200 / 10000
4300 / 10000
4400 / 10000
4500 / 10000
4600 / 10000
4700 / 10000
4800 / 10000
4900 / 10000
5000 / 10000
5100 / 10000
5200 / 10000
5300 / 10000
5400 / 10000
5500 / 10000
5600 / 10000
5700 / 10000
5800 / 10000
5900 / 10000
6000 / 10000
6100 / 10000
6200 / 10000
6300 / 10000
6400 / 10000
6500 / 10000
6600 / 10000
6700 / 10000
6800 / 10000
6900 / 10000
7000 / 10000
7100 / 10000
7200 / 10000
7300 / 10000
7400 / 10000
7500 / 10000
7600 / 10000
7700 / 10000
7800 / 1

In [75]:
for user in random_users:
    dfuser = df.loc[df['user_id'] == users[user]]
    dfuser = dfuser.sort_values('listen_count',ascending=False)
    best_songs_user = list(map(lambda x: show_song(all_songs,x),list(dfuser['song_id'])))
    if len(best_songs_user) > 3:
        print("\n The user likes: \n", best_songs_user[0:3])
    else:
        print("\n The user likes: \n",best_songs_user) 
    recommendations = [y for (x,y) in  getRecommendedItems(itemsim_sim_distance,users[user],df)]
    show = list(map(lambda x: show_song(all_songs,x), recommendations[0:5]))
    print("And we recommend for him/her: \n", show)



 The user likes: 
 [('Eternal Flame (Single Version)', 'Atomic Kitten'), ('Représente', 'Alliance Ethnik'), ("Le Jardin d'Hiver", 'Jacky Terrasson')]
And we recommend for him/her: 
 [('Replay', 'Iyaz'), ('Minnie The Moocher', 'The Blues Brothers'), ('Eple', 'Röyksopp'), ('Deathzone', 'Apocalyptica'), ('Romeo And Juliet', 'The Killers')]

 The user likes: 
 [('More Than A Feeling', 'Boston'), ('Evil', 'Interpol'), ('Hoisting The Flag', 'Sondre Lerche')]
And we recommend for him/her: 
 [('Bullets', 'Creed'), ('Your Place In The World', 'The Space Brothers'), ('Rainy Days And Mondays', 'Carpenters'), ('You Sang To Me', 'Marc Anthony'), ("Let's Go (Produced by B.R. Gunna)", 'Slum Village')]

 The user likes: 
 [('Revelry', 'Kings Of Leon'), ('Undo', 'Björk'), ('Somebody To Love', 'Justin Bieber')]
And we recommend for him/her: 
 [('Razor', 'Foo Fighters'), ('Cool Dry Place (2007 Remastered LP Version)', 'Traveling Wilburys'), ('Be Good Or Be Gone', 'Fionn Regan'), ('Odalisque', 'The Decem

In [76]:
def calculateSimilarItems(itemPrefs,n=10):
    result = {}
    c = 0
    for item in itemPrefs:
        c += 1
        if c%100==0:
            print('%d / %d' % (c,len(itemPrefs)))
        scores = topMatches(itemPrefs, item, n=n, similarity=naive_pearson)
        result[item]=scores
    return result

start_time = time.time()

itemsim_naive_pearson = calculateSimilarItems(dic_songs)

duration = time.time() - start_time
sec = timedelta(seconds=int(duration))
d_time = datetime(1,1,1) + sec    
print("\n duration:  %d:%d:%d:%d" % (d_time.day-1, d_time.hour, d_time.minute, d_time.second), end='')
print(" (DAYS:HOURS:MIN:SEC)") 

100 / 10000
200 / 10000
300 / 10000
400 / 10000
500 / 10000
600 / 10000
700 / 10000
800 / 10000
900 / 10000
1000 / 10000
1100 / 10000
1200 / 10000
1300 / 10000
1400 / 10000
1500 / 10000
1600 / 10000
1700 / 10000
1800 / 10000
1900 / 10000
2000 / 10000
2100 / 10000
2200 / 10000
2300 / 10000
2400 / 10000
2500 / 10000
2600 / 10000
2700 / 10000
2800 / 10000
2900 / 10000
3000 / 10000
3100 / 10000
3200 / 10000
3300 / 10000
3400 / 10000
3500 / 10000
3600 / 10000
3700 / 10000
3800 / 10000
3900 / 10000
4000 / 10000
4100 / 10000
4200 / 10000
4300 / 10000
4400 / 10000
4500 / 10000
4600 / 10000
4700 / 10000
4800 / 10000
4900 / 10000
5000 / 10000
5100 / 10000
5200 / 10000
5300 / 10000
5400 / 10000
5500 / 10000
5600 / 10000
5700 / 10000
5800 / 10000
5900 / 10000
6000 / 10000
6100 / 10000
6200 / 10000
6300 / 10000
6400 / 10000
6500 / 10000
6600 / 10000
6700 / 10000
6800 / 10000
6900 / 10000
7000 / 10000
7100 / 10000
7200 / 10000
7300 / 10000
7400 / 10000
7500 / 10000
7600 / 10000
7700 / 10000
7800 / 1

In [77]:
for user in random_users:
    dfuser = df.loc[df['user_id'] == users[user]]
    dfuser = dfuser.sort_values('listen_count',ascending=False)
    best_songs_user = list(map(lambda x: show_song(all_songs,x),list(dfuser['song_id'])))
    if len(best_songs_user) > 3:
        print("\n The user likes: \n", best_songs_user[0:3])
    else:
        print("\n The user likes: \n",best_songs_user) 
    recommendations = [y for (x,y) in  getRecommendedItems(itemsim_naive_pearson,users[user],df)]
    show = list(map(lambda x: show_song(all_songs,x), recommendations[0:5]))
    print("And we recommend for him/her: \n", show)


 The user likes: 
 [('Eternal Flame (Single Version)', 'Atomic Kitten'), ('Représente', 'Alliance Ethnik'), ("Le Jardin d'Hiver", 'Jacky Terrasson')]
And we recommend for him/her: 
 [('Dream On', 'Aerosmith'), ('Civil War', "Guns N' Roses"), ('Savior', 'Rise Against'), ('Realize', 'Colbie Caillat'), ('Sunday Morning', 'Maroon 5')]

 The user likes: 
 [('More Than A Feeling', 'Boston'), ('Evil', 'Interpol'), ('Hoisting The Flag', 'Sondre Lerche')]
And we recommend for him/her: 
 [('Just Like You', 'Three Days Grace'), ('Banquet', 'Bloc Party'), ("Kiss Me_ I'm Sh*tfaced", 'Dropkick Murphys'), ('Raspberry Beret', 'Prince'), ("Jive Talkin' (Album Version)", 'Bee Gees')]

 The user likes: 
 [('Revelry', 'Kings Of Leon'), ('Undo', 'Björk'), ('Somebody To Love', 'Justin Bieber')]
And we recommend for him/her: 
 [("The '59 Sound", 'The Gaslight Anthem'), ('Drunk Kid Catholic', 'Bright Eyes'), ('Do You Wanna', 'The Kooks'), ('Everybody Everybody', 'Black Box'), ('Ordinary', 'Train')]

 The use

In [78]:
f = open('dic_songs.pickle', 'wb')
di ={'dic_songs':dic_songs,'itemsim':itemsim,'itemsim_naive_pearson':itemsim_naive_pearson, 'itemsim_sim_distance':itemsim_sim_distance}
pickle.dump(di,f)
f.close()

In [79]:
ls -lah

total 187M
drwxrwxr-x 4 felipe felipe 4,0K Dez 29 15:04 ./
drwxr-xr-x 4 felipe felipe 4,0K Dez 28 22:55 ../
-rw-rw-r-- 1 felipe felipe  457 Dez 24 11:24 2016movies.csv
-rw-rw-r-- 1 felipe felipe  33K Dez 27 21:52 chap2_collectiv.ipynb
-rw-rw-r-- 1 felipe felipe  55M Dez 29 15:06 dic_songs.pickle
drwxrwxr-x 8 felipe felipe 4,0K Dez 28 19:34 .git/
drwxr-xr-x 2 felipe felipe 4,0K Dez 29 12:09 .ipynb_checkpoints/
-rw-rw-r-- 1 felipe felipe 1,1K Dez 24 08:55 LICENSE
-rw-rw-r-- 1 felipe felipe   74 Dez 24 09:34 .~lock.teste.html#
-rw-rw-r-- 1 felipe felipe 7,2K Dez 24 13:04 pandas_basic0.ipynb
-rw-rw-r-- 1 felipe felipe  63K Dez 24 12:00 pandas_basic1.ipynb
-rw-rw-r-- 1 felipe felipe  28K Dez 24 15:49 pandas_basic2.ipynb
-rw-rw-r-- 1 felipe felipe  58K Dez 25 12:25 pandas_basic3.ipynb
-rw-rw-r-- 1 felipe felipe  25K Dez 25 20:53 pandas_basic4.ipynb
-rw-rw-r-- 1 felipe felipe  47K Dez 26 15:18 pandas_basic_5.ipynb
-rw-rw-r-- 1 felipe felipe  46K Dez 29 15:04 Recommender_draft_one.ipynb
-rw-rw